# Fitting a line to data using MCMC

In this example we will be going over Exercise 6 from [Hogg 2010](http://arxiv.org/abs/1008.4686).  We will be fitting a line to data using a model that rejects outliers using an MCMC sampler.

## Packages being used
We will make use of the following packages:
+ `numpy`: doing math on arrays
+ `scipy`: more useful functions here
+ `emcee`: this does the heavy lifting for the MCMC code
+ `matplotlib`: plot our results
+ `astropy`: useful hist function

## Relevant documentation
Useful documentation:
+ `scipy`: http://docs.scipy.org/doc/scipy/reference/generated/scipy.misc.logsumexp.html
+ `emcee`: http://dan.iel.fm/emcee/current/
+ `matplotlib`: http://matplotlib.org/1.4.0/api/pyplot_summary.html
+ `astropy`: http://docs.astropy.org/en/stable/visualization/histogram.html

In [0]:
import numpy as np
from scipy.misc import logsumexp
from matplotlib import pyplot as plt
import emcee
from astropy.visualization import hist
from astropy.table import Table
%matplotlib notebook

### Read in the data
Frist lets read in the data we will be fitting (`.data` returns the raw data array):

In [0]:
data = Table.read('data.csv', format='ascii.csv')
x = data['x'].data
y = data['y'].data
sy = data['sy'].data

### Plot the data
Lets take a look at our data to see what we are fitting:

In [0]:
plt.figure(1)
plt.errorbar(x, y, sy, ls='None', mfc='k', ms=2, marker='s', ecolor='k')

### Define the model
To use `emcee` we need to define the log(likelihood) function we are trying to maximize.  The likelihood we need for this fit is shown in equation 17 of [Hogg 2010](http://arxiv.org/abs/1008.4686):

$$\mathscr{L} \propto \prod_{i=1}^{N}{\left[\frac{1-P_{b}}{\sqrt{2 \pi \sigma_{yi}^{2}}} \exp{\left(-\frac{[y_{i} - m x_{i} - b]^2}{2 \sigma_{yi}^{2}}\right)} + \frac{P_{b}}{\sqrt{2 \pi [V_{b}+\sigma_{yi}^{2}]}} \exp{\left( -\frac{[y_{i} - Y_{b}]^2}{2[V_{b}+\sigma_{yi}^{2}]} \right)} \right]}$$

where $x_{i}, y_{i}, \sigma_{yi}$ are the data from the `.csv` file, $m, b$ are the slope and intercept of the line we are fitting, $P_b$ is the probability of a point being an outlier, and $Y_{b}, V_{b}$ are the prameters of the distribution those outliers are draw from.  See section 3 of Hogg's paper for a full derivation.

Since we will be using a Bayesian approach to this fitting, we need to define priors for our fit parameters $\theta = \left[ m, b, P_b, Y_b, V_b \right]$.  We will use flat priors in all the parameters except $V_b$ which will be flat on the log.

In [0]:
def lnprior(theta):
    m, b, Pb, Yb, Vb = theta
    if (-10 < m < 10) and (-500 < b < 500) and (0 < Pb < 1) and (-100 < Yb < 1000) and (0 < np.log(Vb) < 13):
        return 0.0
    else:
        return -np.inf

To make life easy, we will first write a general function of the log(likelihood) of a general Gaussian distribution for the rediusals of the data points from a model:

In [0]:
def lnGau(model, y, var):
    residual = (y - model)**2
    return -0.5 * (np.log(2 * np.pi * var) + residual / var)

This makes it easier to write the above equation as a function.  In `emcee` all the data parameters being fit will come in as the frist variable in the log(likelihood) function.  To avoid numerical overflow we will use `scipy`'s `logsumexp` function to define this model:

In [0]:
def lnprob(theta, x, y, sy):
    lp = lnprior(theta)
    if not np.isfinite(lp):
        # if the params are outside the prior range return -inf
        return -np.inf
    m, b, Pb, Yb, Vb = theta
    # probability of the data points in the line model
    lnpf = lnGau(m * x + b, y, sy**2)
    # probability of the data points in the outlier model
    lnpb = lnGau(Yb, y, Vb + sy**2)
    # combine both probabilities with the propper coefficients and sum them up
    lnlike = logsumexp([lnpf, lnpb], b=[[1 - Pb], [Pb]], axis=0).sum()
    return lp + lnlike

### Set up the MCMC sampler
`emcee` uses an ensemble sampling technique where it sends out $N$ walkers to explore the likelihood space and create a sampling from it.  These walkers all need to start in a reasonable part of parameters space:

In [0]:
# number of parameters being fit
ndim = 5
# number of walkers to use (should be > 2*ndim)
nwalkers = 300
# good starting point
p0 = np.array([1, 240, 0.1, 420, 100])
# start each walker in a small ball around this position
pos = [p0 + 1e-4 * np.random.randn(5) for i in range(nwalkers)]

**Note** For the walkers to move around the parameter space the starting positions must all have _finite_ prior probabilities (i.e. `lnprior(pos)` is finitie for all values).

### Run the sampler
Next we have to set up our sampler and run it.  For this fit we will let it run for 1000 steps.

In [0]:
sampler = emcee.EnsembleSampler(nwalkers, ndim, lnprob, args=(x, y, sy))
sampler.run_mcmc(pos, 1000)

### Check for convergence
When that is done running we should check that our walkers have converged on a solution:

In [0]:
plt.figure(2)
plt.subplot(321)
plt.plot(sampler.chain[:, :, 0].T, alpha=0.05, color='k')
plt.ylabel(r'$m$')
plt.xlabel('step')
plt.subplot(322)
plt.plot(sampler.chain[:, :, 1].T, alpha=0.05, color='k')
plt.ylabel(r'$b$')
plt.xlabel('step')
plt.subplot(323)
plt.plot(sampler.chain[:, :, 2].T, alpha=0.05, color='k')
plt.ylabel(r'$P_b$')
plt.xlabel('step')
plt.subplot(324)
plt.plot(sampler.chain[:, :, 3].T, alpha=0.05, color='k')
plt.ylabel(r'$Y_b$')
plt.xlabel('step')
plt.subplot(325)
plt.plot(np.log10(sampler.chain[:, :, 4].T), alpha=0.05, color='k')
plt.ylabel(r'$\log{(V_b)}$')
plt.xlabel('step')
plt.tight_layout()

### Remove the burn-in and find the 1-$\sigma$ cofidence interval
From these plots we can see it took ~400 steps for the MCMC chain to converge, so we will create our final sample from the last half of the chain.  Taking the median of these value will give us our best fit params, and taking the middle 68% will give us our upper and lower confidence intervals:

In [0]:
samples = sampler.chain[:, 400:, :].reshape((-1, ndim))
fits = np.percentile(samples, [16, 50, 84], axis=0)
fits_pm = map(lambda v: (v[1], v[2]-v[1], v[1]-v[0]), zip(*fits))
print(list(fits_pm))

### Plot the marginalized posterior probabilities
With a sampling from our posterior probability distribution we can visualize the full covariance between $m$ and $b$:

In [0]:
plt.figure(3)
plt.hexbin(samples[:, 0], samples[:, 1], gridsize=100, cmap='Greys', extent=[2, 2.6, -40, 90])
plt.xlabel(r'$m$')
plt.ylabel(r'$b$')

And the fully margalized distributions:

In [0]:
plt.figure(4)
plt.subplot(121)
values, bins, patches = hist(samples[:, 0], bins='knuth', histtype='step', color='k', normed=True)
plt.xlabel(r'$m$')
plt.subplot(122)
values, bins, patches = hist(samples[:, 1], bins='knuth', histtype='step', color='k', normed=True)
plt.xlabel(r'$b$')

As a bonus we can also see what fraction of our data points are outliers:

In [0]:
plt.figure(5)
# use astropy's hist function so it can pick optimal bin sizes
values, bins, patches = hist(samples[:, 2], bins='knuth', histtype='step', color='k', normed=True)
plt.xlabel(r'$P_b$')

This idicates there are ~4 outliers in our sample.

### Plot the final fit
Now that we have best fit line (along with a represetitive sample) lets place it our data to see how well it did:

In [0]:
X = np.linspace(0, 300, 500)
# pick 40 random parameter sets from the final sample
idx = np.random.randint(samples.shape[0], size=40)
# plot the data
plt.figure(6)
plt.errorbar(x, y, sy, ls='None', mfc='k', ms=2, marker='s', ecolor='k')
# plot the best fit line
plt.plot(X, fits[1][0] * X + fits[1][1], color='k')
# plot a sample of best fit lines
for i in idx:
    f = samples[i]
    plt.plot(X, f[0] * X + f[1], color='k', alpha=0.1)

As you can see this does a good job of finding the best fit line while rejecting outliers.

# Exercises
1. Repeat the fit but divide all the uncertainty values ($\sigma_{yi}$) by 2, how does the posterior distribution funciton for $P_b$, $m$, and $b$ change?  Is this what you expected?
2. If you wanted to restrict your fits so they all passed thorugh (or very close to) a fixed point $[X_0, Y_0]$ where would this information go?  Modify this code so your fits all pass through the point $[150, 350]$, how does the chnage the posterior for $P_b$.
3. How would you model _intrinisic_ scatter within the data points?